In [5]:
from google.colab import drive
import os
import re
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# 1. SETUP MÔI TRƯỜNG
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Cài đặt thư viện (nếu chưa có)
!pip install vncorenlp -q

# Setup VnCoreNLP
!mkdir -p vncorenlp/models/wordsegmenter
!wget -q -nc https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar -P vncorenlp/
!wget -q -nc https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab -P vncorenlp/models/wordsegmenter/
!wget -q -nc https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr -P vncorenlp/models/wordsegmenter/

from vncorenlp import VnCoreNLP

# Khởi tạo bộ tách từ
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

# Đường dẫn file
DRIVE_FOLDER = "/content/drive/My Drive/NLP Project/Data"
INPUT_FILE = os.path.join(DRIVE_FOLDER, "vnexpress_full_dataset.csv")
OUTPUT_FILE = os.path.join(DRIVE_FOLDER, "vnexpress_processed_vncorenlp_for_phobert.csv")

In [6]:
def clean_text(text):
    """
    Hàm làm sạch dữ liệu chuyên sâu, xử lý các lỗi dính từ đặc thù của tiếng Việt.
    """
    if not isinstance(text, str): return ""

    # 1. XÓA RÁC CÔNG NGHỆ (Link, Email, Hashtag, Username)
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'\S+@\S+', ' ', text)
    text = re.sub(r'(?:instagram|tiktok|facebook|twitter)/[\w.-]+', ' ', text)
    text = re.sub(r'<.*?>', ' ', text)

    # 2. XỬ LÝ DÍNH TỪ (CamelCase) - Vấn đề "MỹBức", "Thái LanViệc"
    text = re.sub(r'(?<=[a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ])(?=[A-ZÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ])', ' ', text)

    # 3. XỬ LÝ DÍNH SỐ (Smart Number Splitting)
    text = re.sub(r'(?<=\d)(?=[a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ])', ' ', text)
    text = re.sub(r'(?<=[a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ])(?=\d)', ' ', text)

    # 4. CHUẨN HÓA KÝ TỰ PHÂN CÁCH
    text = re.sub(r'[\n\r\t]', ' ', text)
    text = re.sub(r'(?<=[a-zA-Z])[-:](?=[a-zA-Z])', ' ', text)

    # 5. CHUẨN HÓA CUỐI CÙNG
    text = text.lower()
    text = re.sub(r'[^\w\sàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ0-9.,?!%]', ' ', text)
    text = re.sub(r'([.,?!])', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def process_text_pipeline(text):
    # Bước 1: Làm sạch kỹ
    text = clean_text(text)
    if not text: return ""

    try:
        # Bước 2: Tách từ bằng VnCoreNLP
        sentences = rdrsegmenter.tokenize(text)

        # Bước 3: Nối lại
        segmented_text = " ".join([" ".join(sentence) for sentence in sentences])
        return segmented_text
    except:
        return ""

In [7]:
try:
    print(f"\n Đang đọc file: '{INPUT_FILE}'...")
    df = pd.read_csv(INPUT_FILE)
    print(f"   - Tổng số bài thô: {len(df)}")

    # 1. Lọc trùng và rỗng
    df.drop_duplicates(subset=['content'], inplace=True)
    df.dropna(subset=['content'], inplace=True)

    # 2. Test thử hàm Clean mới trên các mẫu lỗi bạn đã báo
    print("\n Test nhanh các trường hợp lỗi trước khi chạy full:")
    test_cases = [
        "BrazilThủ môn Gremio",
        "Thái LanViệc đột ngột",
        "chạy 42km",
        "đội U23 Việt Nam",
        "CR7 ghi bàn",
        "xong.Tuy nhiên",
        "Instagram/GeorginaRodriguez"
    ]
    for case in test_cases:
        print(f"   Original: '{case}'  ->  Cleaned: '{clean_text(case)}'")

    # 3. Chạy xử lý toàn bộ
    print("\n Đang chạy xử lý toàn bộ (VnCoreNLP + Regex V5)...")
    tqdm.pandas()
    df['content_processed'] = df['content'].progress_apply(process_text_pipeline)

    # 4. Lọc bài quá ngắn
    df = df[df['content_processed'].str.len() > 50]

    # 5. Mã hóa nhãn
    label_mapping = {label: idx for idx, label in enumerate(df['label'].unique())}
    df['label_id'] = df['label'].map(label_mapping)

    print("\n Thống kê dữ liệu:")
    print(df['label'].value_counts())
    print("Mapping:", label_mapping)

    # 6. Lưu file
    df_final = df[['content_processed', 'label', 'label_id']]
    df_final.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")

    print(f"\n HOÀN TẤT! Dữ liệu sạch đã lưu tại: {OUTPUT_FILE}")

except FileNotFoundError:
    print(f" Lỗi: Không tìm thấy file '{INPUT_FILE}'")


 Đang đọc file: '/content/drive/My Drive/NLP Project/Data/vnexpress_full_dataset.csv'...
   - Tổng số bài thô: 13928

 Test nhanh các trường hợp lỗi trước khi chạy full:
   Original: 'BrazilThủ môn Gremio'  ->  Cleaned: 'brazil thủ môn gremio'
   Original: 'Thái LanViệc đột ngột'  ->  Cleaned: 'thái lan việc đột ngột'
   Original: 'chạy 42km'  ->  Cleaned: 'chạy 42 km'
   Original: 'đội U23 Việt Nam'  ->  Cleaned: 'đội u23 việt nam'
   Original: 'CR7 ghi bàn'  ->  Cleaned: 'cr7 ghi bàn'
   Original: 'xong.Tuy nhiên'  ->  Cleaned: 'xong . tuy nhiên'
   Original: 'Instagram/GeorginaRodriguez'  ->  Cleaned: 'instagram georgina rodriguez'

 Đang chạy xử lý toàn bộ (VnCoreNLP + Regex V5)...


  0%|          | 0/13730 [00:00<?, ?it/s]


 Thống kê dữ liệu:
label
Kinh_doanh            1491
The_gioi              1489
Doi_song              1485
Khoa_hoc_cong_nghe    1472
The_thao              1467
Suc_khoe              1444
Giao_duc              1406
Thu_gian              1224
Xe                    1148
Phap_luat             1104
Name: count, dtype: int64
Mapping: {'The_thao': 0, 'Suc_khoe': 1, 'Giao_duc': 2, 'Phap_luat': 3, 'Kinh_doanh': 4, 'Thu_gian': 5, 'Khoa_hoc_cong_nghe': 6, 'Xe': 7, 'Doi_song': 8, 'The_gioi': 9}

 HOÀN TẤT! Dữ liệu sạch đã lưu tại: /content/drive/My Drive/NLP Project/Data/vnexpress_processed_vncorenlp_for_phobert.csv
